In [3]:
# %% [markdown]
# # Reperimento Dati per la Regione del Sahel (2010-2023)
# In questo notebook utilizziamo l'API di Google Earth Engine per recuperare:
# - **EVI** (MOD13Q1)
# - **LAI** (MOD15A2H)
# - **Temperatura e Radiazione Solare** (ERA5 Daily)
# - **Precipitazioni** (CHIRPS Daily)
# per la regione del Sahel (approssimativamente definita come rettangolo: [-20, 10, 40, 20]).

# %%
import ee
import json

In [8]:

# Inizializza Earth Engine
ee.Initialize(project="g20-hackaton")

# Definisci la geometria della regione del Sahel (rettangolo approssimativo)
sahel = ee.Geometry.Rectangle([-20, 10, 40, 20])

# Definisci l'intervallo di date
start_date = '2010-01-01'
end_date = '2023-12-31'

# Crea una lista degli anni da 2010 a 2023
years = list(range(2010, 2024))

# %% [markdown]
# ## 1. Recupero dell'EVI (MODIS MOD13Q1)
# Selezioniamo il band "EVI" dal prodotto MOD13Q1, filtrando per data e regione.
# Calcoliamo la media annuale per ogni anno.

# %%
def annual_mean_evi(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, 'year')
    image = (ee.ImageCollection('MODIS/006/MOD13Q1')
             .filterDate(start, end)
             .filterBounds(sahel)
             .select('EVI')
             .mean()
             .set('year', year))
    return image

annual_evi = ee.ImageCollection(ee.List(years).map(annual_mean_evi))
print("EVI Collection:")
print(json.dumps(annual_evi.getInfo(), indent=2))

# %% [markdown]
# ## 2. Recupero del LAI (MODIS MOD15A2H)
# Selezioniamo il band "Lai" dal prodotto MOD15A2H e calcoliamo la media annuale.

# %%
def annual_mean_lai(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, 'year')
    image = (ee.ImageCollection('MODIS/006/MOD15A2H')
             .filterDate(start, end)
             .filterBounds(sahel)
             .select('Lai_500m')
             .mean()
             .set('year', year))
    return image

annual_lai = ee.ImageCollection(ee.List(years).map(annual_mean_lai))
print("LAI Collection:")
print(json.dumps(annual_lai.getInfo(), indent=2))

# %% [markdown]
# ## 3. Recupero della Temperatura e Radiazione Solare (ERA5 Daily)
# Usiamo il dataset ERA5 Daily per ottenere la temperatura a 2 metri e la radiazione solare netta di superficie.
# Calcoliamo la media annuale per ciascun parametro.

# %%
def annual_mean_era5(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, 'year')
    image = (ee.ImageCollection('ECMWF/ERA5/DAILY')
             .filterDate(start, end)
             .filterBounds(sahel)
             .select(['mean_2m_air_temperature'])
             .mean()
             .set('year', year))
    return image

annual_era5 = ee.ImageCollection(ee.List(years).map(annual_mean_era5))
print("ERA5 Collection (Temperatura e Radiazione):")
print(json.dumps(annual_era5.getInfo(), indent=2))

# %% [markdown]
# ## 4. Recupero delle Precipitazioni (CHIRPS Daily)
# Utilizziamo il dataset CHIRPS Daily per le precipitazioni. Calcoliamo il totale annuale per ogni anno.

# %%
def annual_precip(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = start.advance(1, 'year')
    image = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
             .filterDate(start, end)
             .filterBounds(sahel)
             .sum()
             .set('year', year))
    return image

annual_precip = ee.ImageCollection(ee.List(years).map(annual_precip))
print("Precipitazioni Annuali:")
print(json.dumps(annual_precip.getInfo(), indent=2))

# %% [markdown]
# ## Esportazione o Visualizzazione dei Dati
# A questo punto, le collezioni annuali sono pronte per essere visualizzate o esportate.
# Ad esempio, per esportare un'immagine annuale di EVI, puoi utilizzare:
#
# ```python
# task = ee.batch.Export.image.toDrive(
#     image=annual_evi.filter(ee.Filter.eq('year', 2010)).first(),
#     description='EVI_2010',
#     folder='EarthEngineExports',
#     scale=500,
#     region=sahel
# )
# task.start()
# ```
#
# In alternativa, puoi visualizzare le immagini direttamente nella Console di Earth Engine oppure utilizzare strumenti come folium o geemap per visualizzarle in Python.

# %%


EVI Collection:
{
  "type": "ImageCollection",
  "bands": [],
  "features": [
    {
      "type": "Image",
      "bands": [
        {
          "id": "EVI",
          "data_type": {
            "type": "PixelType",
            "precision": "double",
            "min": -32768,
            "max": 32767
          },
          "crs": "EPSG:4326",
          "crs_transform": [
            1,
            0,
            0,
            0,
            1,
            0
          ]
        }
      ],
      "properties": {
        "year": 2010,
        "system:index": "0"
      }
    },
    {
      "type": "Image",
      "bands": [
        {
          "id": "EVI",
          "data_type": {
            "type": "PixelType",
            "precision": "double",
            "min": -32768,
            "max": 32767
          },
          "crs": "EPSG:4326",
          "crs_transform": [
            1,
            0,
            0,
            0,
            1,
            0
          ]
        }
      ],
   

In [11]:
import geemap

# Crea la mappa
Map = geemap.Map(center=[15, 10], zoom=4)

# Aggiungi un layer di esempio (MODIS EVI 2022)
dataset = (ee.ImageCollection('MODIS/006/MOD13Q1')
           .select('EVI')
           .filterDate('2022-01-01', '2022-12-31')
           .mean())

vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['blue', 'white', 'green']
}

Map.addLayer(dataset, vis_params, "EVI 2022")
Map

Map(center=[15, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [17]:
# Crea una mappa centrata (lat=15, lon=10)
Map = geemap.Map(center=[15, 10], zoom=4)

# Esempio: Aggiunge un layer Earth Engine (ad esempio EVI da MOD13Q1)
dataset = ee.ImageCollection('MODIS/006/MOD13Q1').select('EVI') \
            .filterDate('2022-01-01', '2022-12-31').mean()

vis_params = {
    'min': 0.0,
    'max': 6000.0,
    'palette': ['blue', 'white', 'green']
}

# Aggiunge il layer alla mappa
Map.addLayer(dataset, vis_params, "EVI 2022")

# Visualizza la mappa (IMPORTANTE: deve essere l'ultima riga nella cella)
Map

Map(center=[15, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [18]:
from IPython.display import display
display(Map)


Map(center=[15, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…